In [388]:
import pandas as pd
import numpy as np

In [389]:
df = pd.read_csv('../data/carsales.csv')

In [390]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23906 entries, 0 to 23905
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Car_id         23906 non-null  object
 1   Date           23906 non-null  object
 2   Customer Name  23905 non-null  object
 3   Gender         23906 non-null  object
 4   Annual Income  23906 non-null  int64 
 5   Dealer_Name    23906 non-null  object
 6   Company        23906 non-null  object
 7   Model          23906 non-null  object
 8   Engine         23906 non-null  object
 9   Transmission   23906 non-null  object
 10  Color          23906 non-null  object
 11  Price ($)      23906 non-null  int64 
 12  Dealer_No      23906 non-null  object
 13  Body Style     23906 non-null  object
 14  Phone          23906 non-null  int64 
 15  Dealer_Region  23906 non-null  object
dtypes: int64(3), object(13)
memory usage: 2.9+ MB


In [391]:
df['Date'] = pd.to_datetime(df['Date'])

In [392]:
count_sellings_per_month = df.groupby(['Date'])['Car_id'].count().reset_index()
count_sellings_per_month.rename(columns={'Car_id': 'Sellings'}, inplace=True)
billing_per_month = df.groupby(['Date'])['Price ($)'].sum().reset_index()
billing_per_month.rename(columns={'Price ($)': 'Billing'}, inplace=True)
sellings_and_billing_per_month = billing_per_month.merge(
    count_sellings_per_month,
    left_on='Date',
    right_on='Date',
    how='inner'
)
sellings_and_billing_per_month['Date'] = sellings_and_billing_per_month['Date'].apply(
    lambda x: f'{x.month}/{x.year}'
)
sellings_and_billing_per_month['Date'] = pd.to_datetime(sellings_and_billing_per_month['Date'], format='%m/%Y')
sellings_and_billing_per_month = sellings_and_billing_per_month.groupby(['Date'])[['Sellings', 'Billing']].sum().reset_index()

In [393]:
sellings_and_billing_per_month['Sellings_Target'] = sellings_and_billing_per_month['Sellings'].shift(-1)
sellings_and_billing_per_month['Billing_Target'] = sellings_and_billing_per_month['Billing'].shift(-1)
sellings_and_billing_per_month.dropna(inplace=True)

In [394]:
display(sellings_and_billing_per_month.shape)
sellings_and_billing_per_month.head()

(23, 5)

,Date,Sellings,Billing,Sellings_Target,Billing_Target
0,2022-01-01,315,8931920,320.0,8795365.0
1,2022-02-01,320,8795365,705.0,19502059.0
2,2022-03-01,705,19502059,800.0,22748867.0
3,2022-04-01,800,22748867,750.0,20608086.0
4,2022-05-01,750,20608086,690.0,19604211.0


In [395]:
sellings_and_billing_per_month['Billing_Per_Selling'] = sellings_and_billing_per_month['Billing'] / sellings_and_billing_per_month['Sellings']
sellings_and_billing_per_month['Sellings_mm3'] = sellings_and_billing_per_month['Sellings'].rolling(3).mean()
sellings_and_billing_per_month['Billing_mm3'] = sellings_and_billing_per_month['Billing'].rolling(3).mean()

sellings_and_billing_per_month.fillna(method='backfill', inplace=True)

C:\Users\alber\AppData\Local\Temp\ipykernel_17308\1196192135.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sellings_and_billing_per_month.fillna(method='backfill', inplace=True)


In [396]:
sellings_and_billing_per_month.head(50)

,Date,Sellings,Billing,Sellings_Target,Billing_Target,Billing_Per_Selling,Sellings_mm3,Billing_mm3
0,2022-01-01,315,8931920,320.0,8795365.0,28355.301587,446.666667,1.240978e+07
1,2022-02-01,320,8795365,705.0,19502059.0,27485.515625,446.666667,1.240978e+07
2,2022-03-01,705,19502059,800.0,22748867.0,27662.495035,446.666667,1.240978e+07
3,2022-04-01,800,22748867,750.0,20608086.0,28436.083750,608.333333,1.701543e+07
4,2022-05-01,750,20608086,690.0,19604211.0,27477.448000,751.666667,2.095300e+07
5,2022-06-01,690,19604211,700.0,19935002.0,28411.900000,746.666667,2.098705e+07
6,2022-07-01,700,19935002,810.0,23631362.0,28478.574286,713.333333,2.004910e+07
7,2022-08-01,810,23631362,1475.0,42218216.0,29174.520988,733.333333,2.105686e+07
8,2022-09-01,1475,42218216,835.0,23991509.0,28622.519322,995.000000,2.859486e+07
9,2022-10-01,835,23991509,1620.0,45389290.0,28732.346108,1040.000000,2.994703e+07


In [397]:
X = sellings_and_billing_per_month.drop(columns=['Date', 'Sellings_Target', 'Billing_Target'])
y_sellings = sellings_and_billing_per_month['Sellings_Target']
y_billing = sellings_and_billing_per_month['Billing_Target']

In [398]:
display(X.head(1))
display(y_sellings.head(1))
display(y_billing.head(1))

,Sellings,Billing,Billing_Per_Selling,Sellings_mm3,Billing_mm3
0,315,8931920,28355.301587,446.666667,1.240978e+07


0    320.0
Name: Sellings_Target, dtype: float64

0    8795365.0
Name: Billing_Target, dtype: float64

# Model

In [399]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [400]:
# Função que avalia um modelo fazendo divisões em treino e teste na base de dados
# E retornando a média do "mse" e média do "mae"
def test_model_with_kfold(model, Xb, yb) -> dict: 
    kfold = KFold(10)
    metrics = {
        'mse': [],
        'mae': [],
        }
    for train_index, test_index in kfold.split(Xb):
        sellings_model = model()
        sellings_model.fit(Xb[train_index], yb[train_index])
        predicts = sellings_model.predict(Xb[test_index])
        mse = mean_squared_error(yb[test_index], predicts)
        mae = mean_absolute_error(yb[test_index], predicts)
        metrics['mse'].append(np.sqrt(mse))
        metrics['mae'].append(mae)

    metrics['mse'] = np.mean(metrics['mse'])
    metrics['mae'] = np.mean(metrics['mae'])
    return metrics

#### Vamos avaliar o modelo para a previsão dos números de vendas do mês seguinte

In [401]:
# Aqui vamos efetuar o treinamento diversas vezes
# E coletar uma média de "mse" e "mae", fazemos isso pois a base de dados é pequena demais
sellings_metrics = {
    'mse': [],
    'mae': []
}
for _ in range(27):
    result = test_model_with_kfold(RandomForestRegressor, X.values, y_sellings.values)
    sellings_metrics['mse'].append(result['mse'])
    sellings_metrics['mae'].append(result['mae'])
for k, v in sellings_metrics.items():
    sellings_metrics[k] = (np.mean(sellings_metrics[k]), np.std(sellings_metrics[k]))
sellings_metrics

{'mse': (434.4921278796629, 6.938549302828919),
 'mae': (414.3888024691358, 6.936445886110576)}

#### Vamos avaliar o modelo para a previsão do faturamento de vendas do mês seguinte

In [402]:
# Fazendo a mesmo teste para billing
billing_metrics = {
    'mse': [],
    'mae': []
}
for _ in range(27):
    result = test_model_with_kfold(RandomForestRegressor, X.values, y_billing.values)
    billing_metrics['mse'].append(result['mse'])
    billing_metrics['mae'].append(result['mae'])
for k, v in billing_metrics.items():
    billing_metrics[k] = (np.mean(billing_metrics[k]), np.std(billing_metrics[k]))
billing_metrics

{'mse': (12080808.142978989, 289793.42961283715),
 'mae': (11529838.074740741, 290693.77818877227)}

**Após uma análise detalhada na construção dos modelos de previsão, utilizando métricas como o erro quadrático médio (MSE) e o erro absoluto médio (MAE), constatamos que o modelo não obteve a convergência desejada, apesar de aplicarmos procedimentos avançados de engenharia de características. Essa dificuldade de convergência é principalmente atribuída à escassez de dados disponíveis para o treinamento. Para aprimorar a precisão e a confiabilidade do modelo, é crucial aumentar o volume do conjunto de dados, permitindo assim uma representação mais completa e precisa do fenômeno em estudo. Com um conjunto de dados mais amplo e robusto, será possível aperfeiçoar os modelos e suas previsões por meio de um treinamento mais abrangente e preciso.**